In [10]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.optim as optim
import flwr as fl
from flwr.client import ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr.common import Context, ndarrays_to_parameters, parameters_to_ndarrays
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np


In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 5 * 5, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


In [12]:
def get_weights(model):
    return [val.cpu().numpy() for val in model.state_dict().values()]

def set_weights(model, weights):
    state_dict = model.state_dict()
    for k, v in zip(state_dict.keys(), weights):
        state_dict[k] = torch.tensor(v)
    model.load_state_dict(state_dict, strict=True)

def train(model, loader, epochs=1):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    for _ in range(epochs):
        for x, y in loader:
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

def test(model, loader):
    model.eval()
    y_true, y_pred = [], []
    criterion = nn.CrossEntropyLoss()
    loss_total, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in loader:
            out = model(x)
            loss = criterion(out, y)
            loss_total += loss.item()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            y_true.extend(y.numpy())
            y_pred.extend(pred.numpy())
    return loss_total / len(loader), correct / total, y_true, y_pred


In [13]:
transform = transforms.Compose([transforms.ToTensor()])
full_train = MNIST(root=".", train=True, download=True, transform=transform)
testset = MNIST(root=".", train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32)

def partition_dataset(dataset, num_partitions):
    size = len(dataset) // num_partitions
    return [Subset(dataset, list(range(i * size, (i + 1) * size))) for i in range(num_partitions)]

num_clients = 5
partitions = partition_dataset(full_train, num_clients)


In [14]:
class FlowerClient(NumPyClient):
    def __init__(self, cid, trainloader):
        self.model = Net()
        self.trainloader = trainloader
        self.cid = cid

    def get_parameters(self, config): return get_weights(self.model)

    def fit(self, parameters, config):
        set_weights(self.model, parameters)
        train(self.model, self.trainloader)
        return get_weights(self.model), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        set_weights(self.model, parameters)
        loss, acc, _, _ = test(self.model, testloader)
        return loss, len(testloader.dataset), {"accuracy": acc}


In [15]:
def client_fn(context: Context):
    cid = int(context.node_config.get("partition_id", 0))
    trainloader = DataLoader(partitions[cid], batch_size=32, shuffle=True)
    return FlowerClient(cid, trainloader).to_client()

client_app = ClientApp(client_fn)


In [16]:
def server_fn(context: Context) -> ServerAppComponents:
    model = Net()
    initial_parameters = ndarrays_to_parameters(get_weights(model))

    def evaluate_fn(server_round, parameters, _):
        global final_weights
        final_weights = parameters_to_ndarrays(parameters)
        set_weights(model, final_weights)
        loss, acc, _, _ = test(model, testloader)
        print(f"[Server] Round {server_round} - acc: {acc:.4f}")
        return loss, {"accuracy": acc}
    
    def aggregate_fn(results):
        total = sum([num for _, num, _ in results])
        weighted = sum([r[2]['accuracy'] * r[1] for r in results])
        return {"accuracy": weighted / total}


    return ServerAppComponents(
        config=ServerConfig(num_rounds=3),
        strategy=FedAvg(
        initial_parameters=initial_parameters,
        evaluate_fn=evaluate_fn,
        #fit_metrics_aggregation_fn=aggregate_fn
    ),
    )

server = ServerApp(server_fn=server_fn)

In [17]:
history = run_simulation(
    client_app=client_app,
    server_app=server,
    num_supernodes=num_clients,
)


INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters
ERROR :     ServerApp thread raised an exception: 'list' object has no attribute 'tensors'
ERROR :     Traceback (most recent call last):
  File "c:\Users\delma\OneDrive\Bureau\4eme_EI\Artificial_Inteligence\Intership_Steve_DELMAS_2025\venv\Lib\site-packages\flwr\simulation\run_simulation.py", line 292, in server_th_with_start_checks
    run_server_app(
  File "c:\Users\delma\OneDrive\Bureau\4eme_EI\Artificial_Inteligence\Intership_Steve_DELMAS_2025\venv\Lib\site-packages\flwr\server\run_serverapp.py", line 84, in run
    server_app(driver=driver, context=context)
  File "c:\Users\delma\OneDrive\Bureau\4eme_EI\Artificial_Inteligence\Intership_Steve_DELMAS_2025\venv\Lib\site-packages\flwr\server\server_app.py", line 120, in __call__
    start_driver(
  File "c:\Users

RuntimeError: Exception in ServerApp thread

In [ ]:
final_model = Net()
if history and hasattr(history, "parameters") and history.parameters:
    final_parameters = parameters_to_ndarrays(history.parameters[-1])
    set_weights(final_model, final_weights)

    _, _, y_true, y_pred = test(final_model, testloader)
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(range(10)))
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix After Federated Learning")
    plt.show()
else:
    print("⚠️ Erreur : aucune donnée trouvée dans `history.parameters`")

⚠️ Erreur : aucune donnée trouvée dans `history.parameters`
